# MCP - Model Context Protocol

Dokumentacja: https://modelcontextprotocol.io/docs/getting-started/intro

**MCP (Model Context Protocol)** to otwarty standard, który umożliwia łączenie modeli AI z zewnętrznymi systemami, takimi jak pliki, bazy danych, narzędzia i aplikacje.  
Można go porównać do **USB-C dla AI** - jeden standard do wielu połączeń.

### Po co MCP?

- Ujednolica sposób łączenia AI z danymi i narzędziami  
- Skraca czas tworzenia integracji dla developerów  
- Zwiększa możliwości agentów AI (dostęp do realnych danych i akcji)  
- Umożliwia tworzenie bardziej autonomicznych agentów  
- Ułatwia integracje w systemach firmowych  
- Daje użytkownikom bardziej użyteczne i "osobiste" AI  

### Co umożliwia?

- Dostęp do kalendarzy, notatek, baz danych  
- Korzystanie z narzędzi (wyszukiwarki, kalkulatory, API)  
- Automatyzację zadań i workflow  
- Tworzenie aplikacji na podstawie projektów (np. Figma → kod)

### Uczestnicy (Participants)

- **MCP Host**  
  Aplikacja AI (np. Claude Desktop, VS Code), która zarządza połączeniami z serwerami MCP.

- **MCP Client**  
  Komponent w hoście, który utrzymuje **jedno dedykowane połączenie** z jednym MCP serverem.

- **MCP Server**  
  Program, który udostępnia kontekst, narzędzia i dane dla AI (lokalnie lub zdalnie).

> Jeden host może mieć wiele klientów → każdy klient łączy się z innym serwerem.

### Warstwy MCP

MCP składa się z dwóch warstw:

#### 1. Data Layer (warstwa protokołu)

- Oparta na **JSON-RPC 2.0**
- Definiuje:
  - strukturę komunikatów
  - lifecycle połączenia
  - primitives (narzędzia, dane, prompty)
  - notyfikacje

To **najważniejsza warstwa dla developerów**.

#### 2. Transport Layer (warstwa komunikacji)

Odpowiada za sposób przesyłania danych:

- **STDIO**
  - lokalna komunikacja proces–proces  
  - brak narzutu sieciowego  
  - np. lokalny filesystem MCP server  

- **Streamable HTTP**
  - komunikacja przez HTTP + opcjonalny streaming  
  - dla zdalnych serwerów  
  - obsługuje autoryzację (OAuth, API keys, headers)

> Transport nie zmienia formatu komunikatów — zawsze JSON-RPC.

### Primitives MCP

Primitives definiują **co** klient i serwer mogą sobie udostępniać.

#### Serwer → Klient

- **Tools**  
  Funkcje do wykonania (API, DB, pliki, obliczenia)

- **Resources**  
  Źródła danych kontekstowych (pliki, rekordy, odpowiedzi API)

- **Prompts**  
  Gotowe szablony promptów / instrukcji

Każdy primitive ma metody:
- `*/list` – odkrywanie
- `*/get` / `*/read` – pobieranie
- `tools/call` – wykonanie

#### Klient → Serwer

- **Sampling** – serwer prosi klienta o wygenerowanie odpowiedzi z LLM  
- **Elicitation** – serwer prosi użytkownika o dane / potwierdzenie  
- **Logging** – serwer wysyła logi do klienta  

### Dodatkowe mechanizmy

- **Notifications** – serwer wysyła zmiany w czasie rzeczywistym (np. zmiana listy tools)
- **Tasks (eksperymentalne)** – długie operacje z możliwością śledzenia statusu


Przykład: https://modelcontextprotocol.io/docs/learn/architecture#example

Przygotujmy własny MCP server w Pythonie:

In [8]:
%%writefile mcp_server.py

import argparse
import math

from mcp.server.fastmcp import FastMCP

# Initialize FastMCP server - creates an MCP server named "math"
mcp = FastMCP("math")


# ------------------
# TOOLS
# ------------------

# @mcp.tool() decorator exposes this function as an MCP tool
@mcp.tool()
def add(a: float, b: float) -> float:
    """Add two numbers.
    
    Args:
        a: First number
        b: Second number
    """
    return a + b


@mcp.tool()
def subtract(a: float, b: float) -> float:
    """Subtract b from a.
    
    Args:
        a: First number
        b: Second number
    """
    return a - b


@mcp.tool()
def multiply(a: float, b: float) -> float:
    """Multiply two numbers.
    
    Args:
        a: First number  
        b: Second number
    """
    return a * b


@mcp.tool()
def divide(a: float, b: float) -> float:
    """Divide a by b.
    
    Args:
        a: Dividend
        b: Divisor
    """
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b


@mcp.tool()
def power(a: float, b: float) -> float:
    """Raise a to the power of b.
    
    Args:
        a: Base number
        b: Exponent
    """
    return a ** b


@mcp.tool()
def sqrt(a: float) -> float:
    """Calculate square root of a number.
    
    Args:
        a: Number to calculate square root of
    """
    if a < 0:
        raise ValueError("Cannot calculate square root of negative number")
    return a ** 0.5


@mcp.tool()
def factorial(n: int) -> int:
    """Calculate factorial of a non-negative integer.
    
    Args:
        n: Non-negative integer
    """
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    if n == 0 or n == 1:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result


# ------------------
# RESOURCES
# ------------------

@mcp.resource("math://info")
def math_info():
    return "This MCP server provides basic mathematical operations."


@mcp.resource("math://constants/{name}")
def math_constant(name: str):
    constants = {
        "pi": math.pi,
        "e": math.e,
        "tau": math.tau
    }

    name = name.lower()

    if name not in constants:
        return {
            "error": f"Unknown constant: {name}",
            "available": list(constants.keys())
        }

    return {
        "name": name,
        "value": constants[name]
    }


@mcp.resource("math://number/{x}/properties")
def number_properties(x: int):
    return {
        "number": x,
        "even": x % 2 == 0,
        "odd": x % 2 != 0,
        "integer": isinstance(x, int),
        "positive": x > 0,
        "negative": x < 0,
        "zero": x == 0,
    }


# ------------------
# PROMPTS
# ------------------

@mcp.prompt("math_helper")
def math_helper_prompt():
    return """You are a helpful math assistant.
Use available tools to perform calculations accurately.
Explain each step clearly."""


# For ChatGPT
@mcp.tool()
async def search(query: str):
    return {"results": []}


@mcp.tool()
async def fetch(id: str):
    return {"id": id, "title": "Demo", "text": "Demo content", "url": "", "metadata": {}}


# ------------------
# MAIN
# ------------------

def main():
    parser = argparse.ArgumentParser(description="Math MCP Server")
    parser.add_argument(
        "--transport",
        choices=["stdio", "streamable-http", "sse"],
        default="stdio",
        help="Transport type"
    )

    args = parser.parse_args()
    mcp.run(transport=args.transport)


if __name__ == "__main__":
    main()

Overwriting mcp_server.py


Włączymy MCP Inspector

> UWAGA! MCP Inspector wymaga NodeJS.

```bash
npx @modelcontextprotocol/inspector python mcp_server.py --transport streamable-http

npx @modelcontextprotocol/inspector python "/home/adrian/TEG_2025/src/6.MCP/mcp_server.py" --transport streamable-http
```

Docker MCP Catalog: https://docs.docker.com/ai/mcp-catalog-and-toolkit/

## Podłączenie LLM do MCP

Pierwsze systemy, które umożliwiały integrację LLM z MCP, powstały w **Anthropic** i były realizowane w **Claude**.

Uruchommy serwer MCP:

```bash
python ./mcp_server.py --transport streamable-http
```

### ChatGPT

Wykorzystamy Microsoft dev tunniel, aby stworzyć adres routowany z internetu do lokalnego serwera.

Link: https://learn.microsoft.com/en-gb/azure/developer/dev-tunnels/get-started?tabs=linux

```bash
devtunnel host -p 8000 --allow-anonymous
```

W aplikacji ChatGPT: Settings -> Apps -> Advanced settings -> Developer mode (on) -> Back -> Create app

Jako URL wstawić URL z dev tunnel i na końcu `/mcp` (lub zamienić `mcp = FastMCP("math")` na `mcp = FastMCP("math", streamable_http_path="/")`)

![](openai-mcp.png)

**ALE....**

![](openai-mcp-message.png)

### VSCode AI Toolkit plugin

Można zalogować się wykorzystując konto GitHuba lub modele i zasoby na Azure. Można również skonfigurować lokalnie ollama.

1. Z lewej strony w `My Resources` w zakładce Models kliknać `+` i skonfigurować wybrane modele (GitHub, Microsoft Foundry, Ollama).
2. `MCP Servers` - wybrać z listy lub skonfigurować lokalny mcp server poprzez `stdio` lub `http`.
Aby podłączyć wyżej skonfigurowany serwer, wpisz adres http `http://127.0.0.1:8000`
Przykład pliku `mcp.json` z konfiguracją:
```json
{
  "servers": {
    "mcp-mk70d7a2": {
      "type": "http",
      "url": "http://127.0.0.1:8000/mcp",
      "headers": {}
    },
    "playwright": {
      "type": "stdio",
      "command": "npx",
      "args": [
        "@playwright/mcp@latest"
      ]
    }
  }
}
```

> UWAGA: Widoczne są tylko toole z MCP (prompty i resource'y nie sa dostępne)

### Claude Desktop

1. Wejdź w Settings -> Developer -> Edit config

2. Edytuj plik `claude_desktop_config.json`
> UWAGA: Działają tylko serwery w trybie stdio
Przykład pliku konfiguracyjnego:
```json
{
  "mcpServers": {
    "math": {
      "command": "wsl",
      "args": [
        "/home/adrian/TEG_2025/.venv/bin/python",
        "/home/adrian/TEG_2025/src/6.MCP/mcp_server.py",
        "--transport",
        "stdio"
      ]
    }
  }
}
```

3. Zresetuj Claude Desktop

4. W nowym chatcie wybierz `+` -> Connectors i serwery można włączać/wyłączać.

5. Dodatkowa konfiguracja w Ustawieniach w zakładce Connectors.

> UWAGA: Resouurce'y z template jak `@mcp.resource("math://constants/{name}")` nie są widoczne. Pozostałe elementy (prompty, statyczne resource'y) można dodać poprzez `+` -> `Add from <server name>` -> wybierz zasób.

### GitHub Copilot

1. W okienku chatu wybierz ikonkę `Configure Tools...`
2. Wybierz `Add MCP Servers...`
3. Wybierz stdio / http lub inne źródło np. docker.
4. Dany MCP server można dodać na poziomie globalnym/projektu.

Przykładowy plik `.vscode/mcp.json`:

```json
{
  "servers": {
    "my-mcp-server-c974c59d": {
      "type": "stdio",
      "command": "/home/adrian/TEG_2025/.venv/bin/python",
      "args": [
        "/home/adrian/TEG_2025/src/6.MCP/mcp_server.py",
        "--transport",
        "stdio"
      ]
    }
  },
  "inputs": []
}
```

5. W otwartym pliku `mcp.json` kliknąć `Start` -> `More` -> `Browse resources`. W zakładce tej jest możliwość przetestowania resource'ów, również tych z template'ów!

### Agent-to-Agent (A2A) i MCP – obrazowo

**MCP – Model Context Protocol** to jak **USB-C dla LLM**:  
- Standaryzuje sposób podłączania modeli do zewnętrznych narzędzi i serwisów.  
- Dzięki MCP LLM nie musi znać szczegółów implementacji – wystarczy, że serwer obsługuje protokół.  
- Narzędzia MCP mogą być napisane w Pythonie, Node.js, czy innych technologiach.

**Agent-to-Agent (A2A)** to jak **sieć komórkowa dla agentów**:  
- Pozwala agentom komunikować się między sobą w czasie rzeczywistym i współpracować nad zadaniami.  
- Agenci mogą pochodzić z różnych ekosystemów: LangGraph, Foundry, Google itp., tak jak telefon Apple może rozmawiać z Samsungiem.  
- Dzięki A2A jeden agent może poprosić innego o informację lub wykonanie zadania, a odpowiedź trafia z powrotem do inicjatora.

**Podsumowanie:**  
- MCP = uniwersalny kabel/protokół dla narzędzi LLM.  
- A2A = sieć współpracujących agentów różnych "marek", wymieniających dane i zadania.

### Elementy protokołu Agent-to-Agent

**Agent Card** – wizytówka agenta:  
- Zawiera jego **nazwę, identyfikator, opis** i zestaw dostępnych narzędzi.  
- Dzięki niej inne agenty wiedzą, co dany agent potrafi.

**Capability** – zdolności agenta:  
- Definiuje, co agent może zrobić samodzielnie.  
- Mogą to być np. narzędzia MCP, modele LLM, dostęp do API czy własne funkcje obliczeniowe.  
- Capabilities są widoczne dla innych agentów w sieci A2A.

**Executor** – „silnik” agenta:  
- Odpowiada za wykonywanie poleceń i tasków.  
- Wysyła zapytania do MCP tools lub innych agentów i zwraca wyniki.  
- Można myśleć o nim jak o procesorze agenta w sieci komórkowej A2A.

Example: https://www.a2aprotocol.net/docs/a2a-python-sdk-basic

Warto zapoznać się z Docker MCP Hub (beta)

https://hub.docker.com/mcp

https://docs.docker.com/ai/mcp-catalog-and-toolkit/

### Zadanie

Korzystając z dowolnego narzędzia (Claude, GitHub Copilot, VS Code Ai Toolkit, ...) skonfiguruj połączenie z prywatnymi serwisami (np. kalendarz, dysk Google, Notion, GitHub, ...) poprzez serwery MCP. Celem zadania jest stworzenie personalnego asystenta mającego dostęp do danych z zewnątrznych źródeł.